# Scikit-Learn Server


In [1]:
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import joblib
from sklearn import datasets

def main():
    clf = LogisticRegression()
    p = Pipeline([('clf', clf)])
    print('Training model...')
    p.fit(X, y)
    print('Model trained!')

    filename_p = 'model.joblib'
    print('Saving model in %s' % filename_p)
    joblib.dump(p, filename_p)
    print('Model saved!')
    
if __name__ == "__main__":
    print('Loading iris data set...')
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    print('Dataset loaded!')
    main()


Loading iris data set...
Dataset loaded!
Training model...
Model trained!
Saving model in model.joblib
Model saved!


/home/clive/anaconda3/envs/seldon-core/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/clive/anaconda3/envs/seldon-core/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Wrap model using s2i

## REST test

In [3]:
!cd .. && make build_rest

s2i build -E environment_rest ./sklearnserver seldonio/seldon-core-s2i-python37:0.11-SNAPSHOT seldonio/sklearnserver_rest:0.1
---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
Build completed successfully


In [4]:
!docker run --rm -d --name "sklearnserver"  -p 5000:5000 -e PREDICTIVE_UNIT_PARAMETERS='[{"type":"STRING","name":"model_uri","value":"file:///model"}]' -v ${PWD}:/model seldonio/sklearnserver_rest:0.1

8974f8bddcf919052aaefc8e563afb8b37bef38595874355b6868e0fa827a047


Send some random features that conform to the contract

In [5]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:

[[6.297 2.485 5.03  0.09 ]]
RECEIVED RESPONSE:
meta {
}
data {
  ndarray {
    values {
      number_value: 1.0
    }
  }
}




In [6]:
!docker rm sklearnserver --force

sklearnserver


## grpc test

In [7]:
!cd .. && make build_grpc

s2i build -E environment_grpc ./sklearnserver seldonio/seldon-core-s2i-python37:0.11-SNAPSHOT seldonio/sklearnserver_grpc:0.1
---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
Build completed successfully


In [8]:
!docker run --rm -d --name "sklearnserver"  -p 5000:5000 -e PREDICTIVE_UNIT_PARAMETERS='[{"type":"STRING","name":"model_uri","value":"file:///model"}]' -v ${PWD}:/model seldonio/sklearnserver_grpc:0.1

e7eda83721402970145f00541e07a8cc67c9b21bd2d898ec49687bb897441c7e


Test using NDArray payload

In [9]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p --grpc

----------------------------------------
SENDING NEW REQUEST:

[[4.637 4.314 2.837 2.695]]
RECEIVED RESPONSE:
meta {
}
data {
  ndarray {
    values {
      number_value: 0.0
    }
  }
}




Test using Tensor payload

In [10]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p --grpc --tensor

----------------------------------------
SENDING NEW REQUEST:

[[7.991 3.926 8.303 1.303]]
RECEIVED RESPONSE:
meta {
}
data {
  tensor {
    shape: 1
    values: 2.0
  }
}




In [11]:
!docker rm sklearnserver --force

sklearnserver
